In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [4]:
# 데이터 불러오기
data = pd.read_csv("C:/Users/LG/Downloads/diabetes.csv")

In [6]:
# 타겟(BMI) 분리, Outcome 제거
X = data.drop(columns=['Outcome', 'BMI']).values  # 7개 특성
y = data['BMI'].values.astype(np.float32)

In [8]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# 시퀀스 형태로 변환
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [12]:
# BMI를 붙여서 시퀀스 분할
full_data = np.hstack((X, y.reshape(-1, 1)))
X_seq, y_seq = split_sequences(full_data, n_steps=5)

In [14]:
# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)


In [16]:
# Tensor 변환
X_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)  # (batch, channels, seq_len)
X_test = torch.tensor(X_test, dtype=torch.float32).permute(0, 2, 1)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [18]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [24]:
#모델 정의: Conv1D 회귀 모델
class BMICNN(nn.Module):
    def __init__(self):
        super(BMICNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=7, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 5, 64)
        self.fc2 = nn.Linear(64, 1)  # 회귀 출력

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [26]:
#학습 및 평가
model = BMICNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []

# Training loop
for epoch in range(30):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Loss: {train_losses[-1]:.4f}")

Epoch 1, Loss: 1083.0423
Epoch 2, Loss: 864.6016
Epoch 3, Loss: 249.2274
Epoch 4, Loss: 102.4231
Epoch 5, Loss: 79.2588
Epoch 6, Loss: 73.6605
Epoch 7, Loss: 73.7682
Epoch 8, Loss: 67.3138
Epoch 9, Loss: 64.7872
Epoch 10, Loss: 63.3088
Epoch 11, Loss: 61.6138
Epoch 12, Loss: 58.7349
Epoch 13, Loss: 57.7707
Epoch 14, Loss: 57.4317
Epoch 15, Loss: 55.1382
Epoch 16, Loss: 57.0918
Epoch 17, Loss: 55.1739
Epoch 18, Loss: 52.4765
Epoch 19, Loss: 55.0611
Epoch 20, Loss: 52.7251
Epoch 21, Loss: 50.3513
Epoch 22, Loss: 49.2733
Epoch 23, Loss: 51.5007
Epoch 24, Loss: 48.7092
Epoch 25, Loss: 47.5042
Epoch 26, Loss: 47.4314
Epoch 27, Loss: 46.4107
Epoch 28, Loss: 54.0680
Epoch 29, Loss: 48.8829
Epoch 30, Loss: 44.4029


In [27]:
# Evaluation
model.eval()
preds = []
actuals = []
with torch.no_grad():
    for xb, yb in test_loader:
        out = model(xb)
        preds.extend(out.squeeze().cpu().numpy())
        actuals.extend(yb.squeeze().cpu().numpy())

mse = mean_squared_error(actuals, preds)
r2 = r2_score(actuals, preds)
print(f"Test MSE: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

Test MSE: 54.5893
R² Score: 0.1999
